In [23]:
import sys
sys.path += [".."]

In [25]:
import smbh
import numpy as np

# Natural units

## Gravitational constant

$m \rightarrow 10^5 M_{\theta}$

$t \rightarrow 1 \text{ Gyr}$

$d \rightarrow 1 \text{ kpc}$

$$
    G = G_0 \left(\dfrac{1 \text{ kpc}^3}{\left(3.0857\times10^{19}\right)^3  \text{ m}^3}\right)\left(\dfrac{\left(3.154\times10^{16}\right)^2 \text{ s}^2}{1 \text{ Gyr}^2}\right)\left(\dfrac{1.98847\times10^{35} \text{ kg}}{10^5 M_\theta}\right)
$$

In [26]:
from scipy.constants import G as G_0 # m3s-2kg-1

kpc = 3.0856776e19 # in m
gyr = 60 * 60 * 24 * 365.25 * 1e9 # in s
sm = 1.98847e30 # in kg

distance = kpc ** -3
time = gyr ** 2
mass = sm * 1e5 # 1e5 solar masses base unit

G = G_0 * distance * time * mass

print(G)

0.4498489819471222


## Hubble constant

$$
    H = H_0 \left(\dfrac{1 \text{ kpc}}{3.0857\times10^{16} \text{ km}}\right)\left(\dfrac{3.154\times10^{16} \text{ s}}{1 \text{ Gyr}}\right)\left(\dfrac{1 \text{ Mpc}}{1000 \text{ kpc}}\right)
$$

In [27]:
H_0 = 67.66
kpc2 = 1 / (3.0856776e16) # 1 kpc in km
H = H_0 * kpc2 * gyr / 1000
# H = H_0 / 1000

H

0.06919670467193331

# Distributions
## Dark matter density

$$
    \rho_\text{DM}(r) = \dfrac{\rho_0^\text{DM}}{\frac{r}{R_s}\left(1 + \frac{r}{R_s}\right)^2}
$$

```
double darkMatterDensity(double r)
{
    // r += SOFTENING_RADIUS;
    double factor = r / DARK_MATTER_SCALE_RADIUS;
    return DARK_MATTER_DENSITY_0 / (factor * pow(1 + factor, 2));
}
```

## Dark matter cumulative mass

$$
    M_{DM}(r) = 4\pi\rho_0^\text{DM}R_s^3\left[\ln\left(\dfrac{R_s + r}{R_s}\right) - \dfrac{r}{R_s + r}\right] \qquad M_{DM}(R_{vir}) \equiv M_h
$$

```
double darkMatterMass(double r)
{
    double factor = log(1 + r / DARK_MATTER_SCALE_RADIUS) - r / (DARK_MATTER_SCALE_RADIUS + r);
    return 4 * PI * DARK_MATTER_DENSITY_0 * factor * pow(DARK_MATTER_SCALE_RADIUS, 3);
}
```

In [14]:
def darkMatterMass(r):
    global DARK_MATTER_SCALE_RADIUS, DARK_MATTER_DENSITY_0
    factor = log(1 + r / DARK_MATTER_SCALE_RADIUS) - r / (DARK_MATTER_SCALE_RADIUS + r);
    return 4 * PI * DARK_MATTER_DENSITY_0 * factor * pow(DARK_MATTER_SCALE_RADIUS, 3);

## Hernquist density

$$
    \rho_B(r) = \dfrac{M_T^B \mathcal{R}_s}{2\pi r(r + \mathcal{R}_s)^3}
$$

```
double baryonicDensityHernquist(double r)
{
    // r += SOFTENING_RADIUS;
    return BARYONIC_TOTAL_MASS * BARYONIC_SCALE_LENGTH / (2 * PI * r * pow(r + BARYONIC_SCALE_LENGTH, 3));
}
```

In [16]:
def baryonicDensityHernquist(r):
    global PI, BARYONIC_TOTAL_MASS, BARYONIC_SCALE_LENGTH
#     r += SOFTENING_RADIUS
    return BARYONIC_TOTAL_MASS * BARYONIC_SCALE_LENGTH / (2 * PI * r * pow(r + BARYONIC_SCALE_LENGTH, 3))

## Hernquist cumulative mass

$$
    M_B(r) = \dfrac{M_T^B r^2}{(r + \mathcal{R}_s)^2}
$$

```
double baryonicMassHernquist(double r)
{
    return BARYONIC_TOTAL_MASS * pow(r / (r + BARYONIC_SCALE_LENGTH), 2);
}
```

In [17]:
def baryonicMassHernquist(r):
    global BARYONIC_TOTAL_MASS, BARYONIC_SCALE_LENGTH
    return BARYONIC_TOTAL_MASS * pow(r / (r + BARYONIC_SCALE_LENGTH), 2)


$$
    R_{1/2} = \left(1 + \sqrt{2}\right)\mathcal{R}_s
$$

## Constants assigment

### R vir
$$
    \bar{\rho} = \dfrac{M}{V} = \dfrac{M_h}{4/3\pi R_\text{vir}^3} = \dfrac{3M_h}{4\pi R_\text{vir}^3} \equiv 200 \rho_\text{crit} = 200\left(\dfrac{3H^2}{8\pi G}\right)
$$

$$
    \text{R_VIR} = R_\text{vir} = \left(\dfrac{GM_h}{100 H^2}\right)^{1/3} \approx 9.79 \text{ kpc}
$$

In [20]:
HALO_MASS = 1e3
R_VIR = (G * HALO_MASS / (100 * H**2))**(1/3)
R_VIR

9.794117400135734

### DARK_MATTER_SCALE_RADIUS
$$
    \text{DARK_MATTER_SCALE_RADIUS} = R_s = \dfrac{R_\text{vir}}{c}
$$

In [21]:
DARK_MATTER_SCALE_RADIUS = R_VIR / 4
DARK_MATTER_SCALE_RADIUS

2.4485293500339336

### DARK_MATTER_DENSITY_0
To obtain the value of $\rho_0^\text{DM}$, the cumulative dark matter mass was evaluated at $R_\text{vir}$.
$$
    M_h \equiv M_\text{DM}(R_\text{vir}) = 4\pi\rho_0^\text{DM}R_s^3 \left[\ln\left(\dfrac{R_s + c(M_h, z)R_s}{R_s}\right) - \dfrac{c(M_h, z)R_s}{R_s + c(M_h, z)R_s}\right]
$$
$$
    \text{DARK_MATTER_DENSITY_0} = \rho_0^\text{DM} = \dfrac{M_h}{4\pi R_s^3 \left[\ln\left(1 + c(M_h, z)\right) - \dfrac{c(M_h, z)}{1 + c(M_h, z)}\right]}
$$

```
double darkMatterDensity0(double c)
{
    double factor = log(1 + c) - c / (1 + c);
    return HALO_MASS / (4 * PI * pow(DARK_MATTER_SCALE_RADIUS, 3) * factor);
}
```

# Equation of movement

## Choksi et al
$$
    \ddot{x} = \left(-\dfrac{GM_h(x)}{x^2} + a_{DF} - \dot x\dfrac{\dot M_\bullet}{M_\bullet} - qH^2x\right)\hat{x} \qquad \text{as on paper, } M_h(x) = M_{DM}(x)
$$

## Mine
$$
    \ddot{\vec{x}} = \left(-\dfrac{GM(x)}{x^2} - \dot x\dfrac{\dot M_\bullet}{M_\bullet}\right)\hat{x} + \vec{a}_{DF} \qquad M(x) = M_{DM}(x) + M_{B}(x)
$$

